# Loading Dependencies

In [3]:
# Data Manipulation.
from pandas import read_csv

# Data Exploration

In [5]:
# Fetching the data scrapped.
df = read_csv('finalwebscraping.csv')

In [6]:
# Taking a first look on the data.
df

,Unnamed: 0.1,Unnamed: 0,href,Title,Price,RAM,Brand,Hdd,Condition,Seller notes,SSD,Rating,Reviews
0,0,0,https://www.ebay.com/itm/166611579675?epid=132...,Apple Macbook Pro 13.3” 2.5GHz intel Core i5 1...,US $249.00,16 GB,Apple,1 TB,Used: An item that has been used previously. T...,NaN,1 TB,"['4.8', '5.0', '5.0', '5.0']",['I received the MacBook Pro and it was almost...
1,1,1,https://www.ebay.com/itm/285754711451?itmmeta=...,"Apple MacBook Pro 15"" A1286 2.3GHz Core i7 16G...",US $219.00,16 GB,Apple,NaN,Used: An item that has been used previously. T...,NaN,240GB,"['4.9', '5.0', '5.0', '5.0']",['Muy buena computadora. En excelentes condici...
2,2,2,https://www.ebay.com/itm/334577965892?itmmeta=...,Apple Macbook Air 13 (2015) | i5 8GB + 512GB S...,US $246.05/ea,8 GB,Apple,NaN,Used,“2 YEAR WARRANTY INCLUDED!!! LAPTOP ARE FULLY ...,128-512 GB,"['4.8', '5.0', '4.9', '5.0']",['Opened it as soooon as the mail man brought ...
3,3,3,https://www.ebay.com/itm/235462992167?itmmeta=...,SONOMA MacBook Pro 15 RETINA / 4.0GHz QUAD COR...,US $665.00/ea,NaN,Apple,2 TB,Used,“Fully tested and verified! Good condition ove...,2 TB,"['4.9', '5.0', '5.0', '5.0']",['Everything as promised. Fast shipping. Using...
4,4,4,https://www.ebay.com/p/20029930090?iid=2260712...,"Apple MacBook Pro 13"" (128GB SSD, Intel Core i...",US $36.00,NaN,Apple,NaN,NaN,NaN,NaN,[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17663,2223,2223,https://www.ebay.com/itm/296320736271?itmmeta=...,"MacBook Pro 13"" Silver 2017 3.1 GHz i5 7th Gen...",US $254.99,16 GB,Apple,NaN,Used,“Device is 100% fully functional and in Fair p...,256 GB,"['4.9', '5.0', '5.0', '5.0']","['Very good seller, Very fast delivery, very g..."
17664,2224,2224,https://www.ebay.com/itm/404625960187?itmmeta=...,OS SONOMA Apple MacBook Air 13 TOUCH ID Retina...,US $510.72,NaN,Apple,NaN,Used,"“Gray - Industry leading technicians, warranty...",NaN,"['4.9', '5.0', '5.0', '5.0']",['Man!!!!!! This MacBook was waaaaaayyyyyyy be...
17665,2225,2225,https://www.ebay.com/p/28050933187?iid=1261573...,"Apple MacBook Pro Touch Bar 2018 15"" Laptop 6-...",$564.97,NaN,Apple,NaN,NaN,NaN,NaN,[],[]
17666,2226,2226,https://www.ebay.com/itm/305128400646?itmmeta=...,13” Apple MacBook A1181 Laptop Intel Core 2 Du...,US $29.00,1.5 GB,Apple,60 GB,For parts or not working,"“Cosmetic imperfections such as scratches, scu...",NaN,"['4.9', '5.0', '5.0', '5.0']",['Extremely happy with the working condition. ...
